In [1]:
import requests
import pandas as pd
import sqlite3
import time

In [9]:
def compare_strings(s1, s2):
    try:
        s1 = pd.Series(str(s1).lower().split())
        s2 = pd.Series(str(s2).lower().split())
        correct = []
        check = []
        incorrect = []
        
        if len(s1) < len(s2):
            shorter_series, longer_series = s1, s2
        else:
            shorter_series, longer_series = s2, s1
        
        count = shorter_series.isin(longer_series).sum()
        
        if count == len(shorter_series):
            correct.append(shorter_series.tolist())
            return 1
        elif count >= 1:
            check.append(shorter_series.tolist())
            return 2
        else:
            incorrect.append(shorter_series.tolist())
            return 0
    except ValueError as e:
        add_error_to_database(str(e), None, None)
        return None

In [4]:
def add_to_database(compare_result, data):
    conn = sqlite3.connect('database_check.db')
    c = conn.cursor()
    
    c.execute('''
        CREATE TABLE IF NOT EXISTS correct_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            art TEXT,
            query TEXT,
            link_url TEXT,
            brand TEXT,
            name TEXT
        )
    ''')
    c.execute('''
        CREATE TABLE IF NOT EXISTS check_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            art TEXT,
            query TEXT,
            link_url TEXT,
            brand TEXT,
            name TEXT
        )
    ''')
    c.execute('''
        CREATE TABLE IF NOT EXISTS incorrect_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            art TEXT,
            query TEXT,
            link_url TEXT,
            brand TEXT,
            name TEXT
        )
    ''')
    
    if compare_result == 1:
        table_name = 'correct_table'
    elif compare_result == 2:
        table_name = 'check_table'
    else:
        table_name = 'incorrect_table'
    
    query = f"INSERT INTO {table_name} (art, query, link_url, brand, name) VALUES (?, ?, ?, ?, ?)"
    c.execute(query, data)
    
    conn.commit()
    conn.close()

In [5]:
def add_error_to_database(error_message, url, query):
    conn = sqlite3.connect('database_check.db')
    c = conn.cursor()
    
    c.execute('''
        CREATE TABLE IF NOT EXISTS error_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            error_message TEXT,
            url TEXT,
            query TEXT
        )
    ''')
    
    query = "INSERT INTO error_table (error_message, url, query) VALUES (?, ?, ?)"
    c.execute(query, (error_message, url, query))
    
    conn.commit()
    conn.close()

In [6]:
conn = sqlite3.connect('database.db')
c = conn.cursor()
c.execute("SELECT art, query FROM check_table")
results = c.fetchall()

search_arts = [row[0] for row in results]
search_queries = [row[1] for row in results]

In [7]:
for art, query in zip(search_arts, search_queries):
    url = f'https://api.retailrocket.ru/api/2.0/recommendation/Search/525a65850d422d408c357a8c/?&phrase={query}session=64a1b414b2113a5f4a3e6c8d&pvid=605539862619794&isDebug=false&format=json'
    
    response = requests.get(url)
    
    if response.status_code != 200:
        time.sleep(60)
        response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()[0]
        
        try:
            compare_result = compare_strings(query, f"{data['Name']} {data['Params']['name_part_1']}")
            
            if compare_result is not None:
                add_to_database(compare_result, (art, query, data['Url'], data['Params']['brand'], data['Params']['name_part_1']))
        except IndexError as e:
            add_error_to_database(str(e), url, query)
        
    else:
        print(f"Ошибка при выполнении запроса для {query}")